<a href="https://colab.research.google.com/github/abdurrahmanrussel/llama2-qlora-finetune/blob/main/llama2_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upgrade pip and install required packages
!pip install -q -U pip
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U trl
!pip install -q -U datasets safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from getpass import getpass
import os

# ----------------------------
# MODEL / DATA
# ----------------------------
model_name = "NousResearch/Llama-2-7b-chat-hf"  # Hugging Face gated model
dataset_name = "mlabonne/guanaco-llama2-1k"      # Instruction dataset
new_adapter_name = "llama2-7b-qlora-adapter"    # Save folder for LoRA adapter
output_dir = "/content/llama2-qlora-output"     # You can also mount Drive

# ----------------------------
# LoRA params (T4-safe)
# ----------------------------
lora_r = 32
lora_alpha = 16
lora_dropout = 0.05

# ----------------------------
# QLoRA / BitsAndBytes params
# ----------------------------
use_4bit = True
bnb_4bit_quant_type = "nf4"
bnb_4bit_compute_dtype = "float16"
use_double_quant = False

# ----------------------------
# Training params
# ----------------------------
num_train_epochs = 1
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 8  # effective batch = 1*8 = 8
learning_rate = 2e-4
weight_decay = 0.0
gradient_checkpointing = True
max_seq_length = 256  # reduce for T4 GPU
logging_steps = 25
save_steps = 200

effective_batch = per_device_train_batch_size * gradient_accumulation_steps
print(f"Effective batch size = {effective_batch}")

# Device map
device_map = "auto"

# ----------------------------
# Hugging Face token
# ----------------------------
hf_token = getpass("Enter your Hugging Face token: ")
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token


Effective batch size = 8
Enter your Hugging Face token: ··········


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer


In [4]:
print("Loading dataset:", dataset_name)
dataset = load_dataset(dataset_name, split="train")
print(dataset)

# Automatic text field detection
text_field = None
for col in dataset.column_names:
    if col in ("text", "content", "instruction", "prompt", "messages", "input"):
        text_field = col
        break
if text_field is None:
    # fallback to first string column
    for col in dataset.column_names:
        if dataset.features[col].dtype == 'string' or str(dataset.features[col]).startswith('Value'):
            text_field = col
            break
print("Using text field:", text_field)


Loading dataset: mlabonne/guanaco-llama2-1k


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 1000
})
Using text field: text


In [6]:
# ----------------------------
# Step 5 - Load tokenizer + 4-bit model
# ----------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training

# Compute dtype mapping
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_double_quant,
)

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

# Load 4-bit base model
print("Loading 4-bit model (this may take a few minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
)

model.config.use_cache = False

# Prepare for k-bit training (LoRA)
model = prepare_model_for_kbit_training(model)


Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [8]:
# ----------------------------
# Step 6 - Attach LoRA adapters (fixed)
# ----------------------------
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],  # fix for LLaMA-2
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.4f}%)")


Trainable params: 16,777,216 / 3,517,190,144 (0.4770%)


In [14]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments (HF standard)
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_strategy="steps",
    save_steps=save_steps,
    fp16=(bnb_4bit_compute_dtype == "float16"),
    bf16=False,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    group_by_length=True,
    report_to="none",
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:201: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
# ----------------------------
# Step 8 - Train
# ----------------------------
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.510700
50,1.453400
75,1.328100
100,1.351800
125,1.405100


TrainOutput(global_step=125, training_loss=1.4098065185546875, metrics={'train_runtime': 1734.768, 'train_samples_per_second': 0.576, 'train_steps_per_second': 0.072, 'total_flos': 1.675299775500288e+16, 'train_loss': 1.4098065185546875, 'epoch': 1.0})

In [16]:
# ----------------------------
# Step 9 - Save adapter
# ----------------------------
import os

adapter_save_path = os.path.join(output_dir, new_adapter_name)
model.save_pretrained(adapter_save_path)
print("Adapter saved to:", adapter_save_path)

# Optional: merge into full model (bigger)
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(os.path.join(output_dir, "llama2-7b-merged"))


Adapter saved to: /content/llama2-qlora-output/llama2-7b-qlora-adapter


In [17]:
# ----------------------------
# Step 10 - Load adapter + run inference
# ----------------------------
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Reload base model in 4-bit
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
base.config.use_cache = True

# Load LoRA adapter
model_with_adapter = PeftModel.from_pretrained(base, adapter_save_path)

# Prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Prompt example
prompt = "Write a polite email asking for a meeting about QLoRA improvements."

inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model_with_adapter.device)
outputs = model_with_adapter.generate(**inputs, max_length=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Write a polite email asking for a meeting about QLoRA improvements. sierp 2018
Hello [Name],
I hope you are doing well. I am reaching out to you to ask if you would be willing to meet with me to discuss some potential improvements to the QLoRA system. I am particularly interested in improving the system's user interface and user experience, as well as enhancing its functionality to better support the needs of our users.
I would greatly appreciate the opportunity to discuss these ideas with you in more detail. If you are interested, please let me know a time and date that works for you, and I will make sure to schedule the meeting accordingly.
Thank you for your time and consideration.
Best regards,
[Your Name]
